# 1 ЧАСТЬ

Задание №1.

Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

In [2]:
import pandas as pd

In [4]:
ratings = pd.read_csv('u.data.csv', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
movies = pd.read_csv('u.item.csv', sep='|', encoding='latin-1',
                      names=['movie_id','movie_title', 'release_date', 'video_release_date',
                             'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                             'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
                             'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                             'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
movies.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Задание №2. 

Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок



In [51]:
top_user = ratings['user_id'].value_counts().head(1)
top_user.index[0]

405

Задание №3. 

Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

In [55]:
rat_top_user = ratings[ratings.user_id == top_user.index[0]]
rat_top_user.drop(columns = ['user_id'],axis = 1, inplace=True)
rat_top_user.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,item_id,rating,timestamp
12276,56,4,885544911
12383,592,1,885548670
12430,1582,1,885548670
12449,171,1,885549544
12460,580,1,885547447


Задание №4.

Для построения модели нам нужны признаки. В качестве таковых будем использовать:
- Год выхода
- Жанры
- Общее количество оценок
- Суммарную оценку

Добавьте к датафрейму из Задания №3 столбцы:
- По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет
- столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей


In [59]:
# таблица с кол-вом оценок по каждому фильму
score_count_df = ratings['item_id'].value_counts().to_frame()
score_count_df.reset_index(inplace= True)
score_count_df.columns = ['item_id', 'score_count']
score_count_df.head()

,item_id,score_count
0,50,583
1,258,509
2,100,508
3,181,507
4,294,485


In [53]:
# таблица с суммами рейтингов по каждому фильму

sum_rating_df = ratings.groupby('item_id')['rating'].sum().to_frame()
sum_rating_df.reset_index(inplace= True)
sum_rating_df.columns = ['item_id', 'sum_rating']
sum_rating_df.head()

,item_id,sum_rating
0,1,1753
1,2,420
2,3,273
3,4,742
4,5,284


In [67]:
# объединяем таблицы

df_merge = pd.merge(score_count_df, sum_rating_df, how='inner', left_on='item_id', right_on='item_id')
df_merge.head()

,item_id,score_count,sum_rating
0,50,583,2541
1,258,509,1936
2,100,508,2111
3,181,507,2032
4,294,485,1531


In [68]:
# соединяем таблицу из Задания №3 с таблицей df_merge

df_merge = pd.merge(rat_top_user, df_merge, how='inner', left_on='item_id', right_on='item_id')
df_merge.head()

,item_id,rating,timestamp,score_count,sum_rating
0,56,4,885544911,394,1600
1,592,1,885548670,9,30
2,1582,1,885548670,1,1
3,171,1,885549544,65,252
4,580,1,885547447,32,108


In [69]:
# и добавляем данные из таблицы movies

df_merge = pd.merge(df_merge, movies, how='inner', left_on='item_id', right_on='movie_id')
df_merge.drop(columns = ['movie_id', 'timestamp', 'movie_title', 'video_release_date', 'IMDb_URL'],axis = 1, inplace=True)
df_merge.head()

,item_id,rating,score_count,sum_rating,release_date,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,56,4,394,1600,01-Jan-1994,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,592,1,9,30,01-Jan-1995,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1582,1,1,1,01-Jan-1947,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,171,1,65,252,01-Jan-1991,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,580,1,32,108,01-Jan-1995,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [73]:
# приведем данные по году в столбце release_date

df_merge['release_date'] = pd.to_datetime(df_merge['release_date'])
df_merge.loc[:, 'release_date'] = df_merge.loc[:, 'release_date'].dt.year
df_merge.head()

,item_id,rating,score_count,sum_rating,release_date,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,56,4,394,1600,1970,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,592,1,9,30,1970,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1582,1,1,1,1970,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,171,1,65,252,1970,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,580,1,32,108,1970,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Задание №5.

Сформируйте X_train, X_test, y_train, y_test

In [75]:
df_merge.columns

Index(['item_id', 'rating', 'score_count', 'sum_rating', 'release_date',
       'unknown', 'Action', 'Adventure', 'Animation', 'Children's', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [76]:
# сформируем X и y

X, y = df_merge[['score_count', 'sum_rating', 'release_date',
       'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western']], df_merge.rating

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
# сформируем данные на данные для обучения и проверки

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Задание №6.

Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [79]:
from sklearn.linear_model import LinearRegression

In [82]:
# создадим и обучим модель LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

Задание №7.

Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [83]:
from sklearn.metrics import mean_squared_error

In [84]:
mean_squared_error(y_test, model.predict(X_test))

1.053092982446583

# 2 ЧАСТЬ

Задание №8.

Загрузить данные в spark

In [87]:
# установка spark

!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,299 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu 

In [88]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [89]:
!wget -q https://downloads.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop2.7.tgz

In [90]:
!tar -xvf spark-3.2.3-bin-hadoop2.7.tgz

spark-3.2.3-bin-hadoop2.7/
spark-3.2.3-bin-hadoop2.7/LICENSE
spark-3.2.3-bin-hadoop2.7/NOTICE
spark-3.2.3-bin-hadoop2.7/R/
spark-3.2.3-bin-hadoop2.7/R/lib/
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/DESCRIPTION
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/Meta/
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/Meta/Rd.rds
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/Meta/features.rds
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/Meta/hsearch.rds
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/Meta/links.rds
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/Meta/nsInfo.rds
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/Meta/package.rds
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/NAMESPACE
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/R/
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/R/SparkR
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdb
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdx
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/help/
spark-3.2.3-bin-hadoop2.7/R/lib/SparkR/help/An

In [91]:
!pip install -q findspark

In [92]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop2.7"

In [93]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [124]:
# загружаем данные

df_data = spark.read.csv('u.data.csv', inferSchema=True, header=False, sep='\t')
new_col_names = ['user_id', 'item_id', 'rating', 'timestamp']
df_data = df_data.toDF(*new_col_names)
df_data.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [125]:
df_item = spark.read.csv('u.item.csv', inferSchema=True, header=False, sep='|')
new_col_names = ['movie_id','movie_title', 'release_date', 'video_release_date',
                  'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                  'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
                  'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                  'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
df_item = df_item.toDF(*new_col_names)
df_item.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              n

Задание №9.

Средствами спарка вывести среднюю оценку для каждого фильма

In [115]:
df_data.groupby('item_id').avg('rating').show(5)

+-------+------------------+
|item_id|       avg(rating)|
+-------+------------------+
|    496| 4.121212121212121|
|    471|3.6108597285067874|
|    463| 3.859154929577465|
|    148|          3.203125|
|   1342|               2.5|
+-------+------------------+
only showing top 5 rows



Задание №10.

Посчитайте средствами спарка среднюю оценку для каждого жанра

In [129]:
df_join = df_data.join(df_item, df_data.item_id == df_item.movie_id, 'inner').drop('user_id', 'item_id', 'timestamp', 'movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL')
df_join.show(5)

+------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|rating|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|     3|      0|     0|        0|        0|         0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|     3|      0|     0|        0|        0|         0|     0|    1|          0|    0|      0|        1|     0|      0|      1|      0|     0|       1|  0|      0|
|     1|      0|     0|        0|        0|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|     2|      0|     0

Задание №11.

В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)

In [182]:
df_top5 = df_data.groupby('item_id').count().orderBy('count', ascending=False).limit(5)
df_low5 = df_data.groupby('item_id').count().orderBy('count', ascending=True).limit(5)

In [183]:
df_top5 = df_top5.toPandas()
df_top5

,item_id,count
0,50,583
1,258,509
2,100,508
3,181,507
4,294,485


In [184]:
df_low5 = df_low5.toPandas()
df_low5

,item_id,count
0,1507,1
1,1650,1
2,1460,1
3,1339,1
4,1580,1
